In [9]:
!pip install kagglehub


In [10]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("hugomathien/soccer")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/hugomathien/soccer/versions/10


In [11]:
import os

# Check the downloaded dataset directory
print(os.listdir(path))


['database.sqlite']


In [12]:
import sqlite3
import pandas as pd

# Define the correct database path
db_path = "/root/.cache/kagglehub/datasets/hugomathien/soccer/versions/10/database.sqlite"

# Connect to the database
conn = sqlite3.connect(db_path)

# List all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Available Tables in the Database:")
print(tables)


Available Tables in the Database:
                name
0    sqlite_sequence
1  Player_Attributes
2             Player
3              Match
4             League
5            Country
6               Team
7    Team_Attributes


In [13]:
# Load data from key tables
df_matches = pd.read_sql("SELECT * FROM Match", conn)
df_teams = pd.read_sql("SELECT * FROM Team", conn)
df_players = pd.read_sql("SELECT * FROM Player", conn)
df_league = pd.read_sql("SELECT * FROM League", conn)
df_team_attributes = pd.read_sql("SELECT * FROM Team_Attributes", conn)
df_player_attributes = pd.read_sql("SELECT * FROM Player_Attributes", conn)

# Check first few rows of matches
df_matches.head()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [14]:
print(df_matches.columns.tolist())


['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1', 'home_player_2', 'home_player_

In [15]:
# Selecting relevant features
df_ml = df_matches[[
    'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal',
    'possession', 'shoton', 'shotoff', 'foulcommit', 'card', 'corner', 'cross',
    'B365H', 'B365D', 'B365A'  # Betting odds
]]

# Drop rows with missing values
df_ml = df_ml.dropna()

# Define match outcome as target variable
df_ml['match_result'] = df_ml.apply(lambda row:
                                    1 if row['home_team_goal'] > row['away_team_goal']
                                    else (-1 if row['home_team_goal'] < row['away_team_goal'] else 0), axis=1)

# Drop goal columns since they are now in target
df_ml = df_ml.drop(columns=['home_team_goal', 'away_team_goal'])

# Check the first few rows
df_ml.head()


,home_team_api_id,away_team_api_id,possession,shoton,shotoff,foulcommit,card,corner,cross,B365H,B365D,B365A,match_result
1728,10260,10261,<possession><value><comment>56</comment><event...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,<cross><value><stats><crosses>1</crosses></sta...,1.29,5.5,11.00,0
1729,9825,8659,<possession><value><comment>65</comment><event...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<corner><value><stats><corners>1</corners></st...,<cross><value><stats><crosses>1</crosses></sta...,1.20,6.5,15.00,1
1730,8472,8650,<possession><value><comment>45</comment><event...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,<cross><value><stats><crosses>1</crosses></sta...,5.50,3.6,1.67,-1
1731,8654,8528,<possession><value><comment>50</comment><event...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,<cross><value><stats><crosses>1</crosses></sta...,1.91,3.4,4.20,1
1732,10252,8456,<possession><value><comment>51</comment><event...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,<cross><value><stats><corners>1</corners></sta...,1.91,3.4,4.33,1


In [16]:
import re

# Function to extract numeric values from XML-like strings
def extract_numeric(value):
    if isinstance(value, str):
        match = re.search(r"<comment>(\d+)</comment>|<stats><.*?>(\d+)</.*?></stats>", value)
        if match:
            return int(match.group(1) or match.group(2))  # Extract first available number
    return 0  # Default to 0 if no value found

# Apply function to relevant columns
cols_to_clean = ['possession', 'shoton', 'shotoff', 'foulcommit', 'corner', 'cross']
for col in cols_to_clean:
    df_ml[col] = df_ml[col].apply(extract_numeric)

# Convert betting odds to numeric (if not already)
df_ml[['B365H', 'B365D', 'B365A']] = df_ml[['B365H', 'B365D', 'B365A']].astype(float)

# Check cleaned dataset
df_ml.head()


,home_team_api_id,away_team_api_id,possession,shoton,shotoff,foulcommit,card,corner,cross,B365H,B365D,B365A,match_result
1728,10260,10261,56,1,1,1,<card><value><comment>y</comment><stats><ycard...,1,1,1.29,5.5,11.00,0
1729,9825,8659,65,1,1,1,<card />,1,1,1.20,6.5,15.00,1
1730,8472,8650,45,1,1,1,<card><value><comment>y</comment><stats><ycard...,1,1,5.50,3.6,1.67,-1
1731,8654,8528,50,1,1,1,<card><value><comment>y</comment><stats><ycard...,1,1,1.91,3.4,4.20,1
1732,10252,8456,51,1,1,1,<card><value><comment>y</comment><stats><ycard...,1,1,1.91,3.4,4.33,1


In [17]:
import re

# Function to extract yellow and red card counts
def extract_card_count(value):
    if isinstance(value, str):
        ycard_match = re.search(r"<ycard>(\d+)</ycard>", value)
        rcard_match = re.search(r"<rcard>(\d+)</rcard>", value)
        ycard = int(ycard_match.group(1)) if ycard_match else 0
        rcard = int(rcard_match.group(1)) if rcard_match else 0
        return ycard + (rcard * 2)  # Giving red cards double weight
    return 0  # Default to 0 if no value found

# Apply function to 'card' column
df_ml['card'] = df_ml['card'].apply(extract_card_count)

# Check the cleaned dataset
df_ml.head()


,home_team_api_id,away_team_api_id,possession,shoton,shotoff,foulcommit,card,corner,cross,B365H,B365D,B365A,match_result
1728,10260,10261,56,1,1,1,0,1,1,1.29,5.5,11.00,0
1729,9825,8659,65,1,1,1,0,1,1,1.20,6.5,15.00,1
1730,8472,8650,45,1,1,1,0,1,1,5.50,3.6,1.67,-1
1731,8654,8528,50,1,1,1,0,1,1,1.91,3.4,4.20,1
1732,10252,8456,51,1,1,1,0,1,1,1.91,3.4,4.33,1


In [18]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = df_ml.drop(columns=['match_result'])  # Features
y = df_ml['match_result']  # Target variable

# Splitting dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print dataset shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((11280, 12), (2820, 12), (11280,), (2820,))

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train model
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Predictions
y_pred_logreg = logreg.predict(X_test)

# Evaluation
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))


Logistic Regression Accuracy: 0.5397163120567375
              precision    recall  f1-score   support

          -1       0.52      0.45      0.48       798
           0       0.00      0.00      0.00       713
           1       0.55      0.89      0.68      1309

    accuracy                           0.54      2820
   macro avg       0.35      0.45      0.39      2820
weighted avg       0.40      0.54      0.45      2820



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred_rf = rf.predict(X_test)

# Evaluation
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.48723404255319147
              precision    recall  f1-score   support

          -1       0.47      0.45      0.46       798
           0       0.24      0.15      0.18       713
           1       0.56      0.69      0.62      1309

    accuracy                           0.49      2820
   macro avg       0.42      0.43      0.42      2820
weighted avg       0.45      0.49      0.46      2820



In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Assuming df_ml is already prepared with the correct data

# Selecting features and target
X = df_ml[['home_team_api_id', 'away_team_api_id', 'possession', 'shoton', 'shotoff', 'foulcommit', 'card', 'corner', 'cross', 'B365H', 'B365D', 'B365A']]
y = df_ml['match_result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Adjust labels to ensure they are in the range [0, 1, 2]
y_train = y_train - y_train.min()
y_test = y_test - y_test.min()

# Initialize and train the XGBoost model
xgb = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)
xgb.fit(X_train, y_train)

# Predictions
y_pred_xgb = xgb.predict(X_test)

# Evaluation
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:33:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.4769503546099291
              precision    recall  f1-score   support

           0       0.44      0.38      0.41       793
           1       0.28      0.10      0.15       766
           2       0.52      0.76      0.62      1261

    accuracy                           0.48      2820
   macro avg       0.41      0.42      0.39      2820
weighted avg       0.43      0.48      0.43      2820



In [4]:
import sqlite3
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC  # Or SVC if you need a non-linear kernel
from sklearn.metrics import accuracy_score, classification_report
import kagglehub
import os

# 1. Download the dataset (if not already downloaded)
dataset_path = "/root/.cache/kagglehub/datasets/hugomathien/soccer/versions/10/database.sqlite"  # Check if it exists locally
if not os.path.exists(dataset_path):
    path = kagglehub.dataset_download("hugomathien/soccer")
    db_path = os.path.join(path, "database.sqlite") # Correctly join the path
else:
    db_path = dataset_path  # Use existing path if file is present

# 2. Connect to the database and load data (all the data loading and cleaning steps)
conn = sqlite3.connect(db_path)
df_matches = pd.read_sql("SELECT * FROM Match", conn)
# ... (Load other tables if needed - Team, Player, etc.) ...
conn.close()  # Close the connection after reading

# 3. Feature selection and preprocessing (as you did before)
df_ml = df_matches[[
    'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal',
    'possession', 'shoton', 'shotoff', 'foulcommit', 'card', 'corner', 'cross',
    'B365H', 'B365D', 'B365A'
]]

df_ml = df_ml.dropna()

df_ml['match_result'] = df_ml.apply(lambda row:
    1 if row['home_team_goal'] > row['away_team_goal']
    else (-1 if row['home_team_goal'] < row['away_team_goal'] else 0), axis=1)

df_ml = df_ml.drop(columns=['home_team_goal', 'away_team_goal'])


def extract_numeric(value):
    if isinstance(value, str):
        match = re.search(r"<comment>(\d+)</comment>|<stats><.*?>(\d+)</.*?></stats>", value)
        if match:
            return int(match.group(1) or match.group(2))  # Extract first available number
    return 0  # Default to 0 if no value found

cols_to_clean = ['possession', 'shoton', 'shotoff', 'foulcommit', 'corner', 'cross']
for col in cols_to_clean:
    df_ml[col] = df_ml[col].apply(extract_numeric)

df_ml[['B365H', 'B365D', 'B365A']] = df_ml[['B365H', 'B365D', 'B365A']].astype(float)


def extract_card_count(value):
    if isinstance(value, str):
        ycard_match = re.search(r"<ycard>(\d+)</ycard>", value)
        rcard_match = re.search(r"<rcard>(\d+)</rcard>", value)
        ycard = int(ycard_match.group(1)) if ycard_match else 0
        rcard = int(rcard_match.group(1)) if rcard_match else 0
        return ycard + (rcard * 2)  # Giving red cards double weight
    return 0  # Default to 0 if no value found

df_ml['card'] = df_ml['card'].apply(extract_card_count)


# 4. Define features (X) and target (y)  (NOW df_ml is defined)
X = df_ml.drop(columns=['match_result'])
y = df_ml['match_result']

# 5. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 6. Feature Scaling (Essential for SVMs)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7. Train the SVM model (LinearSVC)
model = LinearSVC(random_state=42, max_iter=10000)  # Adjust max_iter if needed
model.fit(X_train_scaled, y_train)

# 8. Make predictions
y_pred = model.predict(X_test_scaled)

# 9. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

100%|██████████| 32.7M/32.7M [00:00<00:00, 73.4MB/s]

Extracting files...


Accuracy: 0.5386524822695036
              precision    recall  f1-score   support

          -1       0.57      0.39      0.46       798
           0       0.50      0.00      0.01       713
           1       0.53      0.92      0.67      1309

    accuracy                           0.54      2820
   macro avg       0.53      0.44      0.38      2820
weighted avg       0.53      0.54      0.45      2820

